In [ ]:
# import는 cell 분리하는 게 좋습니다.

import requests, bs4
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote

In [ ]:
# 1. URL 파라미터 분리하기.
# Service URL
xmlUrl = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson'

MY_API_KEY='IHSiw9Rjbv66FORTPF6LjH3K%2FFPi5LXPibUqh3dA9GOh%2FMwyMy4YVVCjTIolPwpqACy%2BRr4fc0oo3Jns4w1hgg%3D%3D'


queryParams = '?' + urlencode(   
    {
        quote_plus('serviceKey') : MY_API_KEY,    
        quote_plus('pageNo') : '1',
        quote_plus('numOfRows') : '10', 
        quote_plus('startCreateDt') : '20200310', 
        quote_plus('endCreateDt') : '20201210'
     }
)

print(xmlUrl+queryParams)

response = requests.get(xmlUrl + queryParams).text.encode('utf-8')
xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')

http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson?serviceKey=IHSiw9Rjbv66FORTPF6LjH3K%252FFPi5LXPibUqh3dA9GOh%252FMwyMy4YVVCjTIolPwpqACy%252BRr4fc0oo3Jns4w1hgg%253D%253D&pageNo=1&numOfRows=10&startCreateDt=20200310&endCreateDt=20201210


In [ ]:
xmlUrl = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson'

MY_API_KEY='IHSiw9Rjbv66FORTPF6LjH3K%2FFPi5LXPibUqh3dA9GOh%2FMwyMy4YVVCjTIolPwpqACy%2BRr4fc0oo3Jns4w1hgg%3D%3D'


res_url = xmlUrl+"?serviceKey="+MY_API_KEY+"&pageNo=1&numOfRows=10&startCreateDt=20200310&endCreateDt=20201210"

response = requests.get(res_url).text.encode('utf-8')
xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')

In [ ]:
rows = xmlobj.findAll('item')
columns = rows[0].find_all()

rowList= []
nameList = []
columnList = []

rowsLen = len(rows)
for i in range(0, rowsLen):
    columns = rows[i].find_all()

    columnsLen = len(columns)
    for j in range(0, columnsLen):
        if i == 0:
            nameList.append(columns[j].name)
        eachColumn = columns[j].text
        columnList.append(eachColumn)
    rowList.append(columnList)
    columnList = []   
    
df = pd.DataFrame(rowList, columns=nameList)
df2 = df[['stateDt', 'decideCnt']]
df2 = df2.astype({"stateDt": int, "decideCnt": int})

df2 = df2.rename(columns={"stateDt": "날짜", "decideCnt": "누적확진자"})
df2 = df2.sort_values(by= "날짜", ascending = True)
df2 = df2.set_index("날짜")
df2['일별확진자']=abs(df2['누적확진자'] - df2['누적확진자'].shift(-1))

df2


,누적확진자,일별확진자
날짜,,
20200310,7513,242.0
20200311,7755,114.0
20200312,7869,110.0
20200313,7979,107.0
20200314,8086,76.0
...,...,...
20201206,37546,615.0
20201207,38161,585.0
20201208,38746,686.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df2.to_csv("covidcount.csv")
from google.colab import files
files.download("covidcount.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>